# Datasets and DataLoaders

PyTorch provides a way to create your own data via [`torch.utils.data`](https://pytorch.org/docs/stable/data.html) module.

It allows us to:
- Create custom datasets (finite or infinite)
- Quickly load data in batches using multiple processes
- Pin data to GPU memory for faster transfers

## Map-style datasets

Those are the most common datasets and have a really simple structure:
- Create a class by inheriting from [`torch.utils.data.Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset)
- Override `__init__` (__optionally, if needed__)
- Define `__getitem__(self, index)` method (__responsible for obtaining SINGLE sample__)
- Define `__len__(self)` method (__how many samples are there in total__)

Let's see how this looks in practice:

In [ ]:
import typing

import torch


class DummyDataset(torch.utils.data.Dataset):
    def __init__(self, l: typing.List[typing.Any]):
        self.l = l

    # Not dependent on index
    def __getitem__(self, index):
        return self.l[index]

    def __len__(self):
        return len(self.l)


dataset = DummyDataset([0, 1, 2, 3, 4, 5])
len(dataset), dataset[3]

## Exercise

Create a dataset with `X` data and `y` labels by inheriting from `torch.utils.data.Dataset` (name it `Dataset`).
- Code setup in `__init__(X, y)` (verify that `X` and `y` are of equal length (you can use [`assert`](https://stackoverflow.com/questions/5142418/what-is-the-use-of-assert-in-python) for that)
- In `__getitem__` return `tuple` containing sample and it's respective label defined by `index`
- In `__len__` return how many samples are contained in dataset

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        assert len(X) == len(y), "Data and labels have to be of equal length!"
        self.X = X
        self.y = y

    # Not dependent on index
    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return len(self.y)

## DataLoaders

Please notice, our `torch.utils.data.Dataset` __returns single element__.

> In deep learning, we use batches of data (as much as we can fit onto GPU usually)

Because of that, PyTorch provides another abstraction: `torch.utils.data.DataLoader`:

> DataLoader batches our data so it is easily consumed by neural networks

### Automated batching

Usually, we will go for automated batching which is really simple:

In [ ]:
dataset = Dataset(torch.randn(300, 10), torch.randint(0, 5, size=(300,)))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64)

for (X, y) in dataloader:
    print(X.shape, y.shape)

## More about DataLoader

[`torch.utils.data.DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) has a lot of arguments (all of them optional except providing `Dataset` instance), most important ones being:
- `batch_size: int` - we've seen it above
- `shuffle: bool` - whether dataset should be shuffled before each iteration
- `num_workers: int` - how many workers are used for data loading
- `pin_memory: bool` - whether tensors should be loaded into pinned memory regions which may improve tensor transfer to GPU (__more about that during Hardware lesson__)

> `DataLoader` can only be iterated over, no random access is provided

> Many of `DataLoader` arguments are mutually exclusive and there is more than one way to specify certain things. __Always go for the easiest and shortest option if possible__

## num_workers

> `num_workers` specifies how many processes in parallel will be used to load data

### Not enough workers

- Our __pipeline__ will wait for data while neural network sits idle
- Underutilization
- Longer training times

### Too many workers

- Likely to crash in extreme cases
- May crash after a while (importance of model saving)
- Might make the whole system laggy

### How to find correct amount of workers?

- Start with number of available cores divided by two (`multiprocessing.cpu_count()`), usually is a good default
- Test a little larger, in case of crash continue division by two and repeat

> Details about multiple workers are located [here](https://pytorch.org/docs/stable/data.html#multi-process-data-loading)

## Sampler

> Sampler __generates indices__ which are used to to __get data from `Dataset` contained in `DataLoader`__

This means, we can control __how__ our dataset is actually sampled, which allows us to:
- perform over/undersampling
- sample some examples more often, because:
    - they are more important
    - they are harder for neural network and should be seen more often
    
When we specify `shuffle=True` actually an instance of [`torch.utils.data.RandomSampler`](https://pytorch.org/docs/stable/data.html#torch.utils.data.RandomSampler) is implicitly created.

# Torchvision

`torchvision` is part of PyTorch project and described as:

> __The torchvision package consists of popular datasets, model architectures, and common image transformations for computer vision.__

It has to be installed separately (though it's advised to download it with PyTorch).

> We will focus on MNIST dataset so you can see whole PyTorch data loading pipeline

In [ ]:
import tempfile

import matplotlib.pyplot as plt
import torchvision

with tempfile.TemporaryDirectory() as tmp_dir:
    # Training dataset as torch.utils.data.Dataset instance
    train_dataset = torchvision.datasets.MNIST(
        root=tmp_dir,  # where data is stored
        transform=torchvision.transforms.ToTensor(),  # how each sample will be transformed
        train=True,  # we want training data
        download=True,  # should i download it if it's not already here?
    )

    # Test dataset as torch.utils.data.Dataset instance
    test_dataset = torchvision.datasets.MNIST(
        root=tmp_dir,
        transform=torchvision.transforms.ToTensor(),
        train=False,
    )
    
len(train_dataset), len(test_dataset)

In [ ]:
x = train_dataset[0][0]
print(x.shape)
plt.imshow(x.squeeze().numpy(), cmap="gray")
plt.show()

## Splitting data

Sometimes, we might need to split the data (usually to get validation dataset).

PyTorch provides [`torch.utils.data.random_split`](https://pytorch.org/docs/stable/data.html#torch.utils.data.random_split) for those cases:

In [ ]:
train_dataset, validation_dataset = torch.utils.data.random_split(
    train_dataset, [50000, 10000]
)  # split into 50K training & 10K validation

## Dataloaders

Below, dataloaders are created as a dictionary; you may sometimes see this approach:
- Pros:
    - easier to read (instead of `training_dataloader`)
    - easier to use
- Cons:
    - still a lot of repetition

In [ ]:
BATCH_SIZE = 64

dataloaders = {
    "train": torch.utils.data.DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        pin_memory=torch.cuda.is_available(),
    ),
    "validation": torch.utils.data.DataLoader(
        validation_dataset, batch_size=BATCH_SIZE, pin_memory=torch.cuda.is_available()
    ),
    "test": torch.utils.data.DataLoader(
        test_dataset, batch_size=BATCH_SIZE, pin_memory=torch.cuda.is_available()
    ),
}

# Exercise

> __Create custom `Images` `torch.utils.data.Dataset` with the goal of loading your custom images and it's respective labels.__

First, in order to load an image we will use `pillow` library (__default choice for image loading in `torchvision`__):

In [ ]:
from PIL import Image

img = Image.open(file_path)

Cell below defines `DataGenerator` which will create specified number of random images
with their respective random label:

In [ ]:
import pathlib
from dataclasses import dataclass

import numpy as np
import pandas as pd
from PIL import Image


@dataclass
class DataGenerator:
    n_images: int
    n_classes: int

    def generate(self, path: pathlib.Path) -> pd.DataFrame:
        path.mkdir(parents=True, exist_ok=True) 
        filenames = []
        for i in range(self.n_images):
            img_array = np.random.rand(100, 100, 3) * 255
            filename = (path / str(i)).with_suffix(".png")
            Image.fromarray(img_array.astype("uint8")).convert("RGBA").save(filename)

            filenames.append(filename)

        return pd.DataFrame(
            {
                "class": np.random.randint(
                    0, high=self.n_classes, size=(self.n_images)
                ).tolist(),
            }
        )

## Your task

- __Analyze `DataGenerator` class__. Can you explain `dataclass` and `pathlib` usage?
- Generate random data using `DataGenerator` and save returned `pd.DataFrame`.
- Create `Images` `torch.utils.data.Dataset`. It should have the following methods:
    - `__init__` - taking list of filepaths and `pd.DataFrame` as arguments
    - `from_folder` - __static method__ taking `pathlib.Path` folder object and `pd.DataFrame`, iterating over files contained within it. Those `pathlib.Path` files should be passed to `__init__` in order to create an instance of `Images` object
    - __Check `pathlib` modules documentation for more info!__
    - `__getitem__(self, index)` - returns `tuple` `(image, klass)` based on `index`
- __Add `transform` argument__ (like `torchvision`) and apply it to loaded image
- Verify everything works correctly by using `from_folder` and creating an example `DataLoader`

In [ ]:
import typing

import torch
from PIL import Image


class Files(torch.utils.data.Dataset):
    @classmethod
    def from_folder(
        cls, path: pathlib.Path, df: pd.DataFrame, transform=None, regex: str = "*"
    ) -> "Files":
        files = [file for file in path.glob(regex)]
        return cls(files, df, transform)

    def __init__(
        self, files: typing.List[pathlib.Path], df: pd.DataFrame, transform=None
    ):
        super().__init__()
        self.files = files
        self.df = df
        self.transform = transform

    def __len__(self) -> int:
        return len(self.files)

    def __getitem__(self, index):
        raise NotImplementedError


class Images(Files):
    def __getitem__(self, index) -> typing.Tuple[Image, int]:
        klass = self.df.iloc[index]
        sample = Image.open(self.files[index])
        if self.transform is not None:
            sample = self.transform(sample)

        return sample, klass

## Summary

- In PyTorch we create datasets by inheriting from `torch.utils.data.Dataset`:
    - those return a single item during each iteration
    - really simple & lightweight abstraction
- `torch.utils.DataLoader` takes care of:
    - batching
    - shuffling
    - speed ups when loading data
- __You should always provide your dataset instance to `DataLoader`!__
- `torchvision` is part of PyTorch project and provides:
    - datasets
    - transformations
    - vision related models
- Default `pytorch` requires a lot of boilerplate code but is a battle-tested approach to training neural networks

# Challenges

## Assessment

- what are [IterableDatasets](https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset)? Where those might be used?

## Non-assessment

- check out third party module [torchdata](https://github.com/szymonmaszke/torchdata) for other ways to handle caching and using data transformations of PyTorch's `Dataset`
- go over PyTorch's [data documentation](https://pytorch.org/docs/stable/data.html) and learn a little more about the details of design